In [24]:
import numpy as np

import tensorflow as tf
from tensorflow import keras

#OpenCV
import cv2

In [25]:
#Because open cv uses BGR
RED = (0,0,255)
GREEN=(0,255,0)

In [26]:
#opencv object that detect faces
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

Load Model

In [27]:
#load model for face classification
#model was created in me_not_me_classifier.ipynb notebook
model_name = 'face_classifier_ResNet152.h5'
expression_model ='face_expressions_model.h5'

face_classifier = keras.models.load_model(f'models/{model_name}')
expression_model= keras.models.load_model(f'models/{expression_model}')
class_names = ['me', 'not_me']
expression_classes=['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [36]:
def get_extended_image(img, x, y, w, h, k=0.1):
    '''
    Function, that return cropped image from 'img'
    If k=0 returns image, cropped from (x, y) (top left) to (x+w, y+h) (bottom right)
    If k!=0 returns image, cropped from (x-k*w, y-k*h) to (x+k*w, y+(1+k)*h)
    After getting the desired image resize it to 250x250.
    And converts to tensor with shape (1, 250, 250, 3)
    Parameters:
        img (array-like, 2D): The original image
        x (int): x coordinate of the upper-left corner
        y (int): y coordinate of the upper-left corner
        w (int): Width of the desired image
        h (int): Height of the desired image
        k (float): The coefficient of expansion of the image

    Returns:
        image (tensor with shape (1, 250, 250, 3))
    '''

    # The next code block checks that coordinates will be non-negative
    # (in case if desired image is located in top left corner)
    if x - k*w > 0:
        start_x = int(x - k*w)
    else:
        start_x = x
    if y - k*h > 0:
        start_y = int(y - k*h)
    else:
        start_y = y

    end_x = int(x + (1 + k)*w)
    end_y = int(y + (1 + k)*h)

    face_image = img[start_y:end_y,
                     start_x:end_x]
    face_image = tf.image.resize(face_image, [250, 250])
    # shape from (250, 250, 3) to (1, 250, 250, 3)
    face_image = np.expand_dims(face_image, axis=0)
    
    result = face_classifier.predict(face_image)
    prediction = class_names[np.array(
    result[0]).argmax(axis=0)]  # predicted class
    confidence = np.array(result[0]).max(axis=0)  # degree of confidence
    return prediction, confidence
    
    

In [37]:
def get_expression_image(img, x, y, w, h, k=0.1):
    face_roi = img[y:y+h, x:x+w]
    gray_face = cv2.cvtColor(face_roi, cv2.COLOR_BGR2GRAY)
    gray_face_resized = cv2.resize(gray_face, (48, 48))
    gray_face_array = np.expand_dims(np.expand_dims(gray_face_resized / 255.0, axis=0), axis=-1)
    expression_prediction = expression_model.predict(gray_face_array)
    expression_label = expression_classes[np.argmax(expression_prediction)]
    return expression_label

Streaming cycle

In [42]:
video_capture = cv2.VideoCapture(0)  # webcamera

if not video_capture.isOpened():
    print("Unable to access the camera")
else:
    print("Access to the camera was successfully obtained")

print("Streaming started - to quit press ESC")
while True:

    # Capture frame-by-frame
    ret, frame = video_capture.read()
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.3,
        minNeighbors=5,
        minSize=(100, 100),
        flags=cv2.CASCADE_SCALE_IMAGE
    )

    for (x, y, w, h) in faces:
        # for each face on the image detected by OpenCV
        # get extended image of this face
        me_not_me_prediction, confidence = get_extended_image(frame, x, y, w, h, 0.5)

        # classify face and draw a rectangle around the face
        # green for positive class and red for negative
       

        expression_label = get_expression_image(frame,x,y,w,h,0.5)
        
        
        if me_not_me_prediction == 'me':
            color = GREEN
        else:
            color = RED
        # draw a rectangle around the face
        cv2.rectangle(frame,
                      (x, y),  # start_point
                      (x+w, y+h),  # end_point
                      color,
                      2)  # thickness in px
        cv2.putText(frame,
                    # text to put
                    "{:6} - {:.2f}% | {:6}".format(me_not_me_prediction, confidence*100,expression_label),
                    (x, y),
                    cv2.FONT_HERSHEY_PLAIN,  # font
                    1,  # fontScale
                    color,
                    2)  # thickness in px

    # display the resulting frame
    cv2.imshow("Face detector - to quit press ESC", frame)

    # Exit with ESC
    key = cv2.waitKey(1)
    if key % 256 == 27:  # ESC code
        break


# when everything done, release the capture
video_capture.release()
cv2.destroyAllWindows()
print("Streaming ended")

Access to the camera was successfully obtained
Streaming started - to quit press ESC
1/1 [==============================] - 0s 20ms/step
Streaming ended


In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load models
me_not_me_model = load_model("models/face_classifier_ResNet152.h5")
expression_model = load_model("models/face_expressions_model.h5")

# Class labels
me_not_me_classes = ['me', 'not_me']
expression_classes = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

# Colors for drawing
GREEN = (0, 255, 0)
RED = (0, 0, 255)

# Face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Preprocessing function for "Me/Not Me" model
def get_extended_image(img, x, y, w, h, k=0.1):
    if x - k * w > 0:
        start_x = int(x - k * w)
    else:
        start_x = x
    if y - k * h > 0:
        start_y = int(y - k * h)
    else:
        start_y = y

    end_x = int(x + (1 + k) * w)
    end_y = int(y + (1 + k) * h)

    face_image = img[start_y:end_y, start_x:end_x]
    face_image = tf.image.resize(face_image, [250, 250])
    face_image = np.expand_dims(face_image / 255.0, axis=0)  # Normalize and add batch dimension
    return face_image

# Initialize video capture
video_capture = cv2.VideoCapture(0)

if not video_capture.isOpened():
    print("Unable to access the camera")
else:
    print("Access to the camera was successfully obtained")

print("Streaming started - to quit press ESC")

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (640, 480))

while True:
    ret, frame = video_capture.read()
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.3,
        minNeighbors=5,
        minSize=(100, 100),
        flags=cv2.CASCADE_SCALE_IMAGE
    )

    for (x, y, w, h) in faces:
        # Get extended image for "Me/Not Me"
        face_image_me_not_me = get_extended_image(frame, x, y, w, h, 0.5)

        # Classify "Me/Not Me"
        result_me_not_me = me_not_me_model.predict(face_image_me_not_me)
        prediction_me_not_me = me_not_me_classes[np.argmax(result_me_not_me[0])]
        confidence_me_not_me = np.max(result_me_not_me[0])

        # Determine color based on "Me/Not Me" prediction
        if prediction_me_not_me == 'me':
            color = GREEN
        else:
            color = RED

        # Preprocess face for expression model
        

        # Draw rectangle and labels
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame,
                    f"{prediction_me_not_me} ({confidence_me_not_me*100:.1f}%) | {expression_label}",
                    (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    out.write(frame)
    cv2.imshow("Face detector - to quit press ESC", frame)

    # Exit with ESC
    key = cv2.waitKey(1)
    if key % 256 == 27:  # ESC code
        break

# Release resources
video_capture.release()
out.release()
cv2.destroyAllWindows()
print("Streaming ended")


Access to the camera was successfully obtained
Streaming started - to quit press ESC
1/1 [==============================] - 0s 19ms/step
Streaming ended
